In [2]:
import pandas as pd

df_train_origin = pd.read_csv('../data/tesla/train.csv')
df_test = pd.read_csv('../data/tesla/test.csv')

df_train_origin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1353 entries, 0 to 1352
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       1353 non-null   object 
 1   Open       1353 non-null   float64
 2   High       1353 non-null   float64
 3   Low        1353 non-null   float64
 4   Close      1353 non-null   float64
 5   Volume     1353 non-null   int64  
 6   Adj Close  1353 non-null   float64
dtypes: float64(5), int64(1), object(1)
memory usage: 74.1+ KB


In [3]:
n_train = int(len(df_train_origin)*0.8)
df_train = df_train_origin.iloc[:n_train].copy()
df_valid = df_train_origin.iloc[n_train:].copy()
df_train

,Date,Open,High,Low,Close,Volume,Adj Close
0,2010-06-29,19.000000,25.000000,17.540001,23.889999,18766300,23.889999
1,2010-06-30,25.790001,30.420000,23.299999,23.830000,17187100,23.830000
2,2010-07-01,25.000000,25.920000,20.270000,21.959999,8218800,21.959999
3,2010-07-02,23.000000,23.100000,18.709999,19.200001,5139800,19.200001
4,2010-07-06,20.000000,20.000000,15.830000,16.110001,6866900,16.110001
...,...,...,...,...,...,...,...
1077,2014-10-08,260.100006,262.880005,252.639999,259.279999,5055100,259.279999
1078,2014-10-09,262.250000,265.540009,254.399994,257.010010,7361300,257.010010
1079,2014-10-10,244.639999,245.889999,235.199997,236.910004,12888300,236.910004
1080,2014-10-13,238.570007,238.960007,221.000000,224.589996,11268700,224.589996


In [4]:
df_train['Date'] = pd.to_datetime(df_train['Date'])

target_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
features_columns = []

for col in target_columns:
    for i in range(1, 4):
        df_train[f'{col}_lag_{i}'] = df_train[col].shift(i)
        features_columns.append(f'{col}_lag_{i}')

df_train.dropna()

X_train = df_train[features_columns]
Y_train = df_train[target_columns]

X_train.shape, Y_train.shape

((1082, 18), (1082, 6))

In [5]:
df_valid['Date'] = pd.to_datetime(df_valid['Date'])

target_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
features_columns = []

for col in target_columns:
    for i in range(1, 4):
        df_valid[f'{col}_lag_{i}'] = df_valid[col].shift(i)
        features_columns.append(f'{col}_lag_{i}')

df_valid.dropna()

X_valid = df_valid[features_columns]
Y_valid = df_valid[target_columns]

X_valid.shape, Y_valid.shape

((271, 18), (271, 6))

In [7]:
df_test['Date'] = pd.to_datetime(df_test['Date'])

target_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

features_columns = []

for col in target_columns:
    for i in range(1, 4):
        df_test[f'{col}_lag_{i}'] = df_test[col].shift(i)
        features_columns.append(f'{col}_lag_{i}')

df_test.dropna()

X_test = df_test[features_columns]
Y_test = df_test[target_columns]

X_test.shape, Y_test.shape

((339, 18), (339, 6))

In [10]:
df_train_origin['Date'] = pd.to_datetime(df_train_origin['Date'])

target_columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

features_columns = []

for col in target_columns:
    for i in range(1, 4):
        df_train_origin[f'{col}_lag_{i}'] = df_train_origin[col].shift(i)
        features_columns.append(f'{col}_lag_{i}')

df_train_origin.dropna()

X_train_origin = df_train_origin[features_columns]
Y_train_origin = df_train_origin[target_columns]
X_train_origin.shape, Y_train_origin.shape

((1353, 18), (1353, 6))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_absolute_percentage_error

list_depth=[i*10 for i in range(1, 51)]
best_r2_score = -float('inf')
best_depth = None
best_model = None

for depth in list_depth:
    model = DecisionTreeRegressor(max_depth=depth)
    model.fit(X_train, Y_train)
    Y_pred_valid = model.predict(X_valid)
    current_r2_score = r2_score(Y_valid, Y_pred_valid)
    print(f"Depth: {depth}, R2 Score: {current_r2_score}")
    if current_r2_score > best_r2_score:
        best_r2_score = current_r2_score
        best_depth = depth
        best_model = model

best_depth, best_r2_score, best_model

Depth: 10, R2 Score: -0.14669766032639456
Depth: 20, R2 Score: -0.5529970853808716
Depth: 30, R2 Score: -0.055882965992024525
Depth: 40, R2 Score: 0.0730460620504318
Depth: 50, R2 Score: 0.010391590148558264
Depth: 60, R2 Score: -0.09086548881675488
Depth: 70, R2 Score: -0.042269281426296844
Depth: 80, R2 Score: -0.4448027701740074
Depth: 90, R2 Score: -0.36504242696865274
Depth: 100, R2 Score: 0.1606806119584345
Depth: 110, R2 Score: 0.19982715206786772
Depth: 120, R2 Score: 0.15663539956625377
Depth: 130, R2 Score: 0.19223338979465235
Depth: 140, R2 Score: 0.20438912445341906
Depth: 150, R2 Score: -0.19183684325646885
Depth: 160, R2 Score: -0.11677668845310724
Depth: 170, R2 Score: -0.3655102559955578
Depth: 180, R2 Score: 0.12782284902708466
Depth: 190, R2 Score: -0.2635134945414576
Depth: 200, R2 Score: 0.15947082080091127
Depth: 210, R2 Score: 0.05711622494392551
Depth: 220, R2 Score: -0.6177529416108607
Depth: 230, R2 Score: 0.1949118437292631
Depth: 240, R2 Score: -0.42762859652

(460, 0.26880414998411395, DecisionTreeRegressor(max_depth=460))

In [11]:
model.fit(X_train_origin, Y_train_origin)
Y_pred = model.predict(X_test)
r2_score(Y_test, Y_pred), mean_absolute_percentage_error(Y_test, Y_pred)

(-0.31239823646327164, 0.13748505180106924)

In [14]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

best_r2_score = -float('inf')
best_depth = None
best_model = None

list_depth=[i*10 for i in range(1, 51)]
for depth in list_depth:
    model = RandomForestRegressor(max_depth=depth)
    model.fit(X_train, Y_train)
    Y_pred_valid = model.predict(X_valid)
    current_r2_score = r2_score(Y_valid, Y_pred_valid)
#nay de test thoy
#    print(f"Depth: {depth}, R2 Score: {current_r2_score}")
    if current_r2_score > best_r2_score:
        best_r2_score = current_r2_score
        best_depth = depth
        best_model = model

best_depth, best_r2_score, best_model

(390, 0.6089081106114288, RandomForestRegressor(max_depth=390))

In [15]:
model.fit(X_train_origin, Y_train_origin)
Y_pred = model.predict(X_test)
r2_score(Y_test, Y_pred), mean_absolute_percentage_error(Y_test, Y_pred)

(0.5409222120273923, 0.09925462207350282)